### Combining standardized datasets



In [ ]:
#pip install pycountry

  Obtaining dependency information for pycountry from https://files.pythonhosted.org/packages/b1/ec/1fb891d8a2660716aadb2143235481d15ed1cbfe3ad669194690b0604492/pycountry-24.6.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 46.0 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [66]:
import pandas as pd
import numpy as np
import pycountry

In [2]:
# read in all datasets

df_eu =  pd.read_csv("../../3. Data Merging/data/eu_startups.csv")
df_api =  pd.read_csv("../../3. Data Merging/data/standard_startups_api_data.csv")
df_scrape=  pd.read_csv("../../3. Data Merging/data/standard_startups_info_websp.csv")
df_wiki =  pd.read_csv("../../3. Data Merging/data/wikidata_standardized_eu_companies.csv")

In [6]:
df_wiki['start_year'] = pd.to_datetime(df_wiki['start_year'], utc=True).dt.year


In [9]:
df_scrape.drop('country_code', axis=1, inplace=True)

In [20]:
aggregation_rules = {
    col: 'first' for col in df_scrape.columns if col != 'industry_tags'
}

aggregation_rules['industry_tags'] = lambda x: ', '.join(x.dropna().astype(str))


df_aggregated = (
    df_scrape
    .groupby('company_name', dropna=False) # Group by 'company_name'
    .agg(aggregation_rules)               # Apply the custom aggregation rules
#    .reset_index()                        # Convert the 'company_name' index back to a column
)

In [21]:
df_aggregated.reset_index(drop=True, inplace=True)

In [22]:
df_api

,country,company_name,city,funding_amount,industry_tags,start_year,status,website,country_code
0,Belgium,FEST COFFEE MISSION SP. Z O.O.,KRAKOW,NaN,NaN,2022.0,NaN,NaN,NaN
1,Belgium,KOON,Tongeren,NaN,NaN,2008.0,NaN,NaN,NaN
2,Belgium,MAS,Bilzen-Hoeselt,NaN,NaN,2015.0,NaN,NaN,NaN
3,Belgium,MPULS MENEN-WERVIK,Menen,NaN,NaN,1968.0,NaN,NaN,NaN
4,Belgium,Nspire,Lummen,NaN,NaN,2011.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1044,Sweden,"""STUDIO CALUFSEN"" HANDELSBOLAG",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1045,Sweden,"""SWEDECAMION"" Handelsbolag",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1046,Sweden,"""Sawa-Eema"" Handelsbolag",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1047,Sweden,"""Skärkälls andelsförening ek.för.""",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
df_api.drop('country_code', axis=1, inplace=True)

### Combine the datasets

In [24]:
all_dataframes = [df_eu, df_wiki, df_aggregated, df_api]

# Stack the DataFrames one under the other
combined_df = pd.concat(all_dataframes, ignore_index=True)

In [ ]:
combined_df['company_name'].isna().sum()

# no misisng company names

np.int64(0)

In [27]:
combined_df.shape

(38725, 8)

All data makes 38.725 observations

In [28]:
# beside company name require 2 more non NA features

minimum_required_non_na = 3

combined_df.dropna(thresh=3, inplace=True)

In [29]:
combined_df.shape

(38442, 8)

Roughly 300 companies are dropped that have less that 3 features present

In [31]:
combined_df.tail(100)

,country,company_name,city,funding_amount,industry_tags,start_year,status,website
38475,Romania,""" CARDIOFORCE "" SRL",Bucureşti Sectorul 4,NaN,NaN,NaN,NaN,NaN
38476,Romania,""" INOVAŢIE"" CENTRUL DE CONSULTAŢII ÎN AFACERI ...",Sat Răteşti,NaN,NaN,NaN,NaN,NaN
38477,Romania,""" LA TASSE "" IMPEX SRL",Bucureşti Sectorul 2,NaN,NaN,NaN,NaN,NaN
38478,Romania,""" REX "" SRL COCORA IALOMITA SUCURSALA BUCURESTI",Bucureşti Sectorul 2,NaN,NaN,NaN,NaN,NaN
38479,Romania,""" SmArt Tree "" Grup SRL",Sat Săftica,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
38570,Slovakia,""" Federmayer - BREAG """,NaN,NaN,NaN,1990.0,NaN,NaN
38571,Slovakia,""" GALIMPEX - S, s.r.o """,NaN,NaN,NaN,1993.0,NaN,NaN
38572,Slovakia,""" GRIMPLINI - stavebné práce s.r.o."" Košice",NaN,NaN,NaN,1995.0,NaN,NaN
38573,Slovakia,""" GROSYMA spol. s r. o. """,NaN,NaN,NaN,1991.0,NaN,NaN


In [ ]:
# process some company names that came with other informations

combined_df['company_name_temp'] = combined_df['company_name'].str.extract(r'"(.*?)"', expand=False)

combined_df['company_name'] = combined_df['company_name_temp'].fillna(combined_df['company_name'])

combined_df = combined_df.drop(columns=['company_name_temp'])

In [35]:
combined_df.tail(100)

,country,company_name,city,funding_amount,industry_tags,start_year,status,website
38475,Romania,CARDIOFORCE,Bucureşti Sectorul 4,NaN,NaN,NaN,NaN,NaN
38476,Romania,INOVAŢIE,Sat Răteşti,NaN,NaN,NaN,NaN,NaN
38477,Romania,LA TASSE,Bucureşti Sectorul 2,NaN,NaN,NaN,NaN,NaN
38478,Romania,REX,Bucureşti Sectorul 2,NaN,NaN,NaN,NaN,NaN
38479,Romania,SmArt Tree,Sat Săftica,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
38570,Slovakia,Federmayer - BREAG,NaN,NaN,NaN,1990.0,NaN,NaN
38571,Slovakia,"GALIMPEX - S, s.r.o",NaN,NaN,NaN,1993.0,NaN,NaN
38572,Slovakia,GRIMPLINI - stavebné práce s.r.o.,NaN,NaN,NaN,1995.0,NaN,NaN
38573,Slovakia,GROSYMA spol. s r. o.,NaN,NaN,NaN,1991.0,NaN,NaN


### Deduplicate

In [36]:
combined_df['company_name'] = combined_df['company_name'].str.strip()

In [37]:
combined_df['company_name_decap'] = combined_df['company_name'].str.lower()

In [38]:
combined_df['company_name_decap'].duplicated().sum()

np.int64(907)

There are 907 duplicate companies

In [51]:
combined_df['non_na_count'] = combined_df.count(axis=1)

df_sorted = combined_df.sort_values(
    by=['company_name_decap', 'non_na_count'],
    ascending=[True, False]
)


df_deduplicated = df_sorted.drop_duplicates(
    subset=['company_name_decap'],
    keep='first'
)

df_deduplicated = df_deduplicated.drop(columns=['non_na_count'])

In [52]:
df_deduplicated.tail(100)

,country,company_name,city,funding_amount,industry_tags,start_year,status,website,company_name_decap
37601,Finland,ZenRobotics,Helsinki,1.589965e+07,,NaN,NaN,NaN,zenrobotics
32724,Netherlands,Zentiva,Amsterdam,NaN,Pharmaceutical Industry,1998.0,NaN,zentiva.cz,zentiva
32725,Switzerland,Zentralbahn,Stansstad,NaN,Rail Transport,2005.0,NaN,zentralbahn.ch,zentralbahn
32731,Denmark,Zentropa,Hvidovre Municipality,NaN,Filmmaking,1992.0,NaN,zentropa.dk,zentropa
33390,Denmark,Zenvo Automotive,Præstø,NaN,Automotive Industry,2007.0,NaN,zenvoautomotive.com,zenvo automotive
...,...,...,...,...,...,...,...,...,...
38169,Greece,Πελίγκος Α. & Σία,ΤΡΙΚΑΛΩΝ,NaN,NaN,1982.0,NaN,NaN,πελίγκος α. & σία
38171,Greece,ΣΚΥΔΡΑ FRUIT ΑΝΩΝΥΜΗ ΕΤΑΙΡΙΑ,ΠΕΛΛΗΣ,NaN,NaN,2015.0,NaN,NaN,σκυδρα fruit ανωνυμη εταιρια
38172,Greece,ΣΥΣΤΕΓΑΣΜΕΝΑ ΦΑΡΜΑΚΕΙΑ ΠΑΝΑΓΙΩΤΗ Γ. ΚΑΡΤΕΡΟΛΙΩ...,ΜΕΣΣΗΝΙΑΣ,NaN,NaN,2018.0,NaN,NaN,συστεγασμενα φαρμακεια παναγιωτη γ. καρτερολιω...
38174,Greece,"ΤΕΤΡΑΣ - ΕΜΠΟΡΙΚΕΣ, ΒΙΟΤΕΧΝΙΚΕΣ, ΒΙΟΜΗΧΑΝΙΚΕΣ ...",ΑΤΤΙΚΗΣ,NaN,NaN,1988.0,NaN,NaN,"τετρας - εμπορικες, βιοτεχνικες, βιομηχανικες ..."


In [53]:
df_deduplicated.drop('company_name_decap', axis=1, inplace=True)

In [54]:
df_deduplicated.head(20)

,country,company_name,city,funding_amount,industry_tags,start_year,status,website
37728,Bulgaria,,област: Стара Загора,NaN,NaN,2012.0,NaN,NaN
38156,Greece,""" Κ. ΠΕΠΠΑΣ-Π. ΣΤΡΑΙΤΟΥΡΗΣ Ο.Ε.",ΔΥΤΙΚΗΣ ΑΤΤΙΚΗΣ,NaN,NaN,NaN,NaN,NaN
37961,Denmark,"""ANPARTSSELSKABET SØNDAG 16.8.1987",Sønderborg,NaN,NaN,1976.0,NaN,NaN
38279,Luxembourg,#BeHuman Luxembourg A.s.b.l.,NaN,NaN,NaN,2015.0,NaN,NaN
38280,Luxembourg,#Luxusbuerg a.s.b.l.,NaN,NaN,NaN,2006.0,NaN,NaN
38281,Luxembourg,#play14 A.s.b.l.,NaN,NaN,NaN,2017.0,NaN,NaN
38005,Finland,& Two Words Oy,NaN,NaN,NaN,2024.0,NaN,NaN
38282,Luxembourg,&coll SARL,NaN,NaN,NaN,2009.0,NaN,NaN
36440,Ireland,&Open,Dublin,6180480.0,"CRM, Product Design, Software",2017.0,NaN,NaN
38329,Malta,''CONFIC'' LTD.,MELLIEHA,NaN,NaN,2011.0,NaN,NaN


In [63]:
df_deduplicated = df_deduplicated[df_deduplicated['company_name'] != '']

### Get countries uniform

In [56]:
df_deduplicated['country'].unique()

array(['Bulgaria', 'Greece', 'Denmark', 'Luxembourg', 'Finland',
       'Ireland', 'Malta', 'Belgium', 'Latvia', 'Italy', 'UK', 'Spain',
       'France', 'Slovenia', 'Romania', 'Germany', 'Czechia',
       'United Kingdom', 'Portugal', 'Cyprus', 'Estonia', 'Switzerland',
       'Czech Republic', 'Netherlands', 'Sweden', 'Lithuania', 'Austria',
       'Norway', 'Slovakia', 'Croatia', 'Poland', 'Hungary',
       'Austria,Germany', 'Germany,Poland', 'Spain,UK', 'Greece,Romania',
       'Estonia,UK', 'France,Luxembourg'], dtype=object)

UK - United Kingdom
Czech Republic - Czechia
some have two countries indicated

In [57]:
country_mapping = {
    'UK': 'United Kingdom',
    'Czech Republic': 'Czechia',
    'Austria,Germany': 'Austria',
    'Germany,Poland': 'Germany',
    'Spain,UK': 'Spain',
    'Greece,Romania': 'Greece',
    'Estonia,UK': 'Estonia',
    'France,Luxembourg': 'France',
}

In [58]:
df_deduplicated['country_standardized'] = df_deduplicated['country'].map(country_mapping)

In [60]:
df_deduplicated['country_standardized'] = df_deduplicated['country_standardized'].fillna(
    df_deduplicated['country']
)

# Optional: Drop the old column and rename the new one
df_deduplicated = df_deduplicated.drop(columns=['country']).rename(
    columns={'country_standardized': 'country'}
)

In [61]:
df_deduplicated['country'].unique()

array(['Bulgaria', 'Greece', 'Denmark', 'Luxembourg', 'Finland',
       'Ireland', 'Malta', 'Belgium', 'Latvia', 'Italy', 'United Kingdom',
       'Spain', 'France', 'Slovenia', 'Romania', 'Germany', 'Czechia',
       'Portugal', 'Cyprus', 'Estonia', 'Switzerland', 'Netherlands',
       'Sweden', 'Lithuania', 'Austria', 'Norway', 'Slovakia', 'Croatia',
       'Poland', 'Hungary'], dtype=object)

In [64]:
df_deduplicated

,company_name,city,funding_amount,industry_tags,start_year,status,website,country
38156,""" Κ. ΠΕΠΠΑΣ-Π. ΣΤΡΑΙΤΟΥΡΗΣ Ο.Ε.",ΔΥΤΙΚΗΣ ΑΤΤΙΚΗΣ,NaN,NaN,NaN,NaN,NaN,Greece
37961,"""ANPARTSSELSKABET SØNDAG 16.8.1987",Sønderborg,NaN,NaN,1976.0,NaN,NaN,Denmark
38279,#BeHuman Luxembourg A.s.b.l.,NaN,NaN,NaN,2015.0,NaN,NaN,Luxembourg
38280,#Luxusbuerg a.s.b.l.,NaN,NaN,NaN,2006.0,NaN,NaN,Luxembourg
38281,#play14 A.s.b.l.,NaN,NaN,NaN,2017.0,NaN,NaN,Luxembourg
...,...,...,...,...,...,...,...,...
38169,Πελίγκος Α. & Σία,ΤΡΙΚΑΛΩΝ,NaN,NaN,1982.0,NaN,NaN,Greece
38171,ΣΚΥΔΡΑ FRUIT ΑΝΩΝΥΜΗ ΕΤΑΙΡΙΑ,ΠΕΛΛΗΣ,NaN,NaN,2015.0,NaN,NaN,Greece
38172,ΣΥΣΤΕΓΑΣΜΕΝΑ ΦΑΡΜΑΚΕΙΑ ΠΑΝΑΓΙΩΤΗ Γ. ΚΑΡΤΕΡΟΛΙΩ...,ΜΕΣΣΗΝΙΑΣ,NaN,NaN,2018.0,NaN,NaN,Greece
38174,"ΤΕΤΡΑΣ - ΕΜΠΟΡΙΚΕΣ, ΒΙΟΤΕΧΝΙΚΕΣ, ΒΙΟΜΗΧΑΝΙΚΕΣ ...",ΑΤΤΙΚΗΣ,NaN,NaN,1988.0,NaN,NaN,Greece


### Get country codes

In [67]:
def get_iso3_code(country_name):

    if pd.isna(country_name) or country_name == '':
        return np.nan
    try:

        country = pycountry.countries.get(name=country_name)

        if country is None:
             country = pycountry.countries.get(common_name=country_name)

        return country.alpha_3

    except AttributeError:

        return np.nan
    except Exception:

        return np.nan

df_deduplicated['country_code'] = df_deduplicated['country'].apply(get_iso3_code)

/tmp/ipykernel_2346/2508208528.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_deduplicated['country_code'] = df_deduplicated['country'].apply(get_iso3_code)


In [68]:
df_deduplicated

,company_name,city,funding_amount,industry_tags,start_year,status,website,country,country_code
38156,""" Κ. ΠΕΠΠΑΣ-Π. ΣΤΡΑΙΤΟΥΡΗΣ Ο.Ε.",ΔΥΤΙΚΗΣ ΑΤΤΙΚΗΣ,NaN,NaN,NaN,NaN,NaN,Greece,GRC
37961,"""ANPARTSSELSKABET SØNDAG 16.8.1987",Sønderborg,NaN,NaN,1976.0,NaN,NaN,Denmark,DNK
38279,#BeHuman Luxembourg A.s.b.l.,NaN,NaN,NaN,2015.0,NaN,NaN,Luxembourg,LUX
38280,#Luxusbuerg a.s.b.l.,NaN,NaN,NaN,2006.0,NaN,NaN,Luxembourg,LUX
38281,#play14 A.s.b.l.,NaN,NaN,NaN,2017.0,NaN,NaN,Luxembourg,LUX
...,...,...,...,...,...,...,...,...,...
38169,Πελίγκος Α. & Σία,ΤΡΙΚΑΛΩΝ,NaN,NaN,1982.0,NaN,NaN,Greece,GRC
38171,ΣΚΥΔΡΑ FRUIT ΑΝΩΝΥΜΗ ΕΤΑΙΡΙΑ,ΠΕΛΛΗΣ,NaN,NaN,2015.0,NaN,NaN,Greece,GRC
38172,ΣΥΣΤΕΓΑΣΜΕΝΑ ΦΑΡΜΑΚΕΙΑ ΠΑΝΑΓΙΩΤΗ Γ. ΚΑΡΤΕΡΟΛΙΩ...,ΜΕΣΣΗΝΙΑΣ,NaN,NaN,2018.0,NaN,NaN,Greece,GRC
38174,"ΤΕΤΡΑΣ - ΕΜΠΟΡΙΚΕΣ, ΒΙΟΤΕΧΝΙΚΕΣ, ΒΙΟΜΗΧΑΝΙΚΕΣ ...",ΑΤΤΙΚΗΣ,NaN,NaN,1988.0,NaN,NaN,Greece,GRC


In [69]:
df_deduplicated['country_code'].isna().sum()

np.int64(0)

### encode status as true false

In [73]:
df_deduplicated['status'].unique()

array([1.0, nan, 0.0], dtype=object)

In [75]:
status_mapping = {
    1.0: True,
    0.0: False
}


df_deduplicated['status'] = df_deduplicated['status'].map(status_mapping)

df_deduplicated.head()

/tmp/ipykernel_2346/3820626709.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_deduplicated['status'] = df_deduplicated['status'].map(status_mapping)


,company_name,city,funding_amount,industry_tags,start_year,status,website,country,country_code
28688,Tagbase,Vienna,300000.0,"Software Development, Product Authenticity, An...",2025.0,True,https://tagbase.io,Austria,AUT
29539,Proofcheck,Vienna,300000.0,"AI, PDFs, Multilingual Machine, Web app",2022.0,True,https://www.proofcheck.io/,Austria,AUT
17549,Repose,Vienna,NaN,"Rental platform, Travel Tech",2017.0,True,www.reposee.com,Austria,AUT
32870,Salzburger Eisenbahn Transportlogistik,Salzburg,NaN,Rail Freight Transport,2001.0,NaN,setg.at,Austria,AUT
872,Streamtv Media,Steyr,1750000.0,"Online Games, Video, Video Streaming",2018.0,True,https://www.stream.tv/,Austria,AUT


### Sort dataset based on country name

In [70]:
df_deduplicated.sort_values(by='country', ascending=True, inplace=True)

/tmp/ipykernel_2346/1914491838.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_deduplicated.sort_values(by='country', ascending=True, inplace=True)


In [76]:
df_deduplicated

,company_name,city,funding_amount,industry_tags,start_year,status,website,country,country_code
28688,Tagbase,Vienna,300000.0,"Software Development, Product Authenticity, An...",2025.0,True,https://tagbase.io,Austria,AUT
29539,Proofcheck,Vienna,300000.0,"AI, PDFs, Multilingual Machine, Web app",2022.0,True,https://www.proofcheck.io/,Austria,AUT
17549,Repose,Vienna,NaN,"Rental platform, Travel Tech",2017.0,True,www.reposee.com,Austria,AUT
32870,Salzburger Eisenbahn Transportlogistik,Salzburg,NaN,Rail Freight Transport,2001.0,NaN,setg.at,Austria,AUT
872,Streamtv Media,Steyr,1750000.0,"Online Games, Video, Video Streaming",2018.0,True,https://www.stream.tv/,Austria,AUT
...,...,...,...,...,...,...,...,...,...
194,Soundly,London,NaN,"e-marketplace, Blockchain, Artificial Intellig...",2020.0,True,https://soundly.art/,United Kingdom,GBR
193,Soundjam,Brighton,NaN,"music, website builder, DJ, eCommerce",2019.0,True,https://www.soundjam.co/,United Kingdom,GBR
22844,Earth 2,"London, England",NaN,"Real Estate, Sales, Virtual Reality, Web Apps",2020.0,True,https://earth2.io,United Kingdom,GBR
20830,Paytrix,London,NaN,"API, Financial Services, software",2022.0,True,https://paytrix.io/,United Kingdom,GBR


In [77]:
df_deduplicated = df_deduplicated.reset_index(drop=True)

In [78]:
df_deduplicated.head(15)

,company_name,city,funding_amount,industry_tags,start_year,status,website,country,country_code
0,Tagbase,Vienna,300000.0,"Software Development, Product Authenticity, An...",2025.0,True,https://tagbase.io,Austria,AUT
1,Proofcheck,Vienna,300000.0,"AI, PDFs, Multilingual Machine, Web app",2022.0,True,https://www.proofcheck.io/,Austria,AUT
2,Repose,Vienna,NaN,"Rental platform, Travel Tech",2017.0,True,www.reposee.com,Austria,AUT
3,Salzburger Eisenbahn Transportlogistik,Salzburg,NaN,Rail Freight Transport,2001.0,NaN,setg.at,Austria,AUT
4,Streamtv Media,Steyr,1750000.0,"Online Games, Video, Video Streaming",2018.0,True,https://www.stream.tv/,Austria,AUT
5,Caterme,Piringsdorf,NaN,"catering, events",2020.0,True,https://catermebyschnabls.com,Austria,AUT
6,Happy Plates,Vienna,1750000.0,"Content Creators, Cooking, E-Commerce, Food De...",2018.0,True,https://happyplates.com,Austria,AUT
7,Vitrealab,Vienna,1750000.0,"Manufacturing, Engineering, Deeptech",2018.0,True,https://vitrealab.com/,Austria,AUT
8,Streemfire,Vienna,NaN,"Internet, Media and Enterainment, Video Streaming",2020.0,True,https://www.streemfire.com/,Austria,AUT
9,Sortask,Graz,300000.0,"Software Development, AI, App",2025.0,True,https://www.sortask.com,Austria,AUT


In [79]:
df_deduplicated.to_csv('../data/combinded_data.csv', index=False)